In [1]:
from langchain_core.messages import AIMessage, BaseMessage
from langgraph.graph import END, START, StateGraph
from typing_extensions import TypedDict


# 定义图的状态
class AgentState(TypedDict):
    messages: list[BaseMessage]
    current_step: str


# 创建图
workflow = StateGraph(AgentState)


# 定义节点函数
def research_node(state: AgentState):
    """研究节点 - 使用工具进行研究"""
    print("执行: 研究节点")
    # 在实际应用中，这里会调用工具或模型
    return {"messages": state["messages"], "current_step": "research_complete"}


def summarize_node(state: AgentState):
    """总结节点 - 总结研究结果"""
    print("执行: 总结节点")
    summary = AIMessage(content="这是研究的总结")
    return {
        "messages": state["messages"] + [summary],
        "current_step": "summary_complete",
    }


def action_node(state: AgentState):
    """行动节点 - 基于研究采取行动"""
    print("执行: 行动节点")
    action = AIMessage(content="基于研究的建议")
    return {"messages": state["messages"] + [action], "current_step": "action_complete"}


# 添加节点到图
workflow.add_node("research", research_node)
workflow.add_node("summarize", summarize_node)
workflow.add_node("action", action_node)

# 添加边来连接节点（定义流程）
workflow.add_edge(START, "research")  # 从开始到研究
workflow.add_edge("research", "summarize")  # 研究完后总结
workflow.add_edge("summarize", "action")  # 总结后采取行动
workflow.add_edge("action", END)  # 行动后结束

# 编译图
graph = workflow.compile()

print("图已创建，节点:")
print(f"- {', '.join(graph.nodes.keys())}")

图已创建，节点:
- __start__, research, summarize, action


In [2]:
from langchain_core.messages import HumanMessage

# 调用图
initial_state = {
    "messages": [HumanMessage(content="请研究并建议")],
    "current_step": "started",
}

result = graph.invoke(initial_state)

print("\n最终状态:")
print(f"当前步骤: {result['current_step']}")
print(f"消息数量: {len(result['messages'])}")

执行: 研究节点
执行: 总结节点
执行: 行动节点

最终状态:
当前步骤: action_complete
消息数量: 3


In [3]:
# 创建带条件边的更复杂的图
workflow2 = StateGraph(AgentState)


def decide_path(state: AgentState):
    """决定应该采取哪个路径"""
    # 根据消息内容决定
    last_message = state["messages"][-1].content.lower() if state["messages"] else ""

    if "urgent" in last_message or "紧急" in last_message:
        return "quick_action"
    else:
        return "normal_process"


def quick_action_node(state: AgentState):
    """快速行动节点"""
    print("执行: 快速行动")
    return {"messages": state["messages"], "current_step": "quick_done"}


def normal_process_node(state: AgentState):
    """正常流程节点"""
    print("执行: 正常流程")
    return {"messages": state["messages"], "current_step": "normal_done"}


# 添加节点
workflow2.add_node(
    "decide", lambda s: {"messages": s["messages"], "current_step": "deciding"}
)
workflow2.add_node("quick_action", quick_action_node)
workflow2.add_node("normal_process", normal_process_node)

# 添加条件边
workflow2.add_edge(START, "decide")

# 条件边：根据 decide_path 的返回值选择路径
workflow2.add_conditional_edges(
    "decide",
    decide_path,
    {"quick_action": "quick_action", "normal_process": "normal_process"},
)

workflow2.add_edge("quick_action", END)
workflow2.add_edge("normal_process", END)

# 编译并测试
graph2 = workflow2.compile()

# 测试案例1：紧急请求
print("测试案例1 - 紧急请求:")
result_urgent = graph2.invoke(
    {
        "messages": [HumanMessage(content="这是紧急的！请立即处理")],
        "current_step": "start",
    }
)
print(f"结果: {result_urgent['current_step']}\n")

# 测试案例2：普通请求
print("测试案例2 - 普通请求:")
result_normal = graph2.invoke(
    {"messages": [HumanMessage(content="请研究这个问题")], "current_step": "start"}
)
print(f"结果: {result_normal['current_step']}")

测试案例1 - 紧急请求:
执行: 快速行动
结果: quick_done

测试案例2 - 普通请求:
执行: 正常流程
结果: normal_done


综合练习 构建完整的聊天代理

现在让我们整合所有概念，构建一个真实的聊天代理。


In [ ]:
from dataclasses import dataclass

from langchain.agents import create_agent
from langchain.tools import ToolRuntime, tool
from langgraph.checkpoint.memory import InMemorySaver

from src.llm.llm import llm

# ==================== 1. 定义数据模型 ====================


@dataclass
class UserContext:
    """用户上下文 - 运行时信息"""

    user_id: str
    user_name: str = "访客"
    interaction_count: int = 0


# ==================== 2. 定义工具 ====================


@tool
def get_user_info(runtime: ToolRuntime[UserContext]) -> str:
    """获取当前用户信息"""
    ctx = runtime.context
    return (
        f"用户: {ctx.user_name} (ID: {ctx.user_id}), 交互次数: {ctx.interaction_count}"
    )


@tool
def search_knowledge(query: str) -> str:
    """搜索知识库"""
    # 模拟知识库搜索
    knowledge_base = {
        "LangChain": "LangChain 是一个用于开发 LLM 应用的框架",
        "LangGraph": "LangGraph 是 LangChain 的持久化执行层",
        "Memory": "记忆系统让代理能记住过去的交互",
    }

    for key, value in knowledge_base.items():
        if query.lower() in key.lower() or query.lower() in value.lower():
            return value

    return f"未找到关于 '{query}' 的信息"


@tool
def log_interaction(action: str) -> str:
    """记录用户交互"""
    from datetime import datetime

    timestamp = datetime.now().strftime("%H:%M:%S")
    return f"[{timestamp}] 记录: {action}"


# ==================== 3. 创建完整的代理 ====================

system_prompt = """你是一个智能助手，帮助用户了解 LangChain 和 AI 开发。

你可以使用以下工具：
- get_user_info: 获取当前用户信息
- search_knowledge: 搜索知识库
- log_interaction: 记录交互日志

根据用户问题：
1. 首先获取用户信息
2. 根据问题搜索相关知识
3. 提供有帮助的回答
4. 记录交互"""

complete_agent = create_agent(
    model=llm,
    tools=[get_user_info, search_knowledge, log_interaction],
    system_prompt=system_prompt,
    checkpointer=InMemorySaver(),
)

# ==================== 4. 测试代理 ====================

# 用户上下文
user_ctx = UserContext(user_id="user_001", user_name="小王")

# 第一轮对话
print("=== 第一轮对话 ===")
response1 = complete_agent.invoke(
    {"messages": [{"role": "user", "content": "你好，请告诉我 LangChain 是什么？"}]},
    {"configurable": {"thread_id": "user_001"}, "context": user_ctx},
)

print("用户问: 你好，请告诉我 LangChain 是什么？")
print(f"助手答: {response1['messages'][-1].content}\n")

# 第二轮对话 - 展示记忆能力
print("=== 第二轮对话（同一线程，展示记忆）===")
response2 = complete_agent.invoke(
    {"messages": [{"role": "user", "content": "那 LangGraph 呢？"}]},
    {"configurable": {"thread_id": "user_001"}, "context": user_ctx},
)

print("用户问: 那 LangGraph 呢？")
print(f"助手答: {response2['messages'][-1].content}\n")

# 查看完整对话历史
print("=== 完整对话历史 ===")
print(f"共有 {len(response2['messages'])} 条消息:")
for i, msg in enumerate(response2["messages"], 1):
    role = "用户" if msg.type == "human" else "助手"
    content = msg.content[:100] + "..." if len(msg.content) > 100 else msg.content
    print(f"{i}. [{role}] {content}")

=== 第一轮对话 ===
用户问: 你好，请告诉我 LangChain 是什么？
助手答: 


=== 第二轮对话（同一线程，展示记忆）===
用户问: 那 LangGraph 呢？
助手答: 
根据我的知识库查询结果，为您介绍这两个概念：

**LangChain**
- LangChain 是一个用于开发 LLM 应用的框架
- 它为开发者提供了构建基于大语言模型的应用所需的工具和组件

**LangGraph**  
- LangGraph 是 LangChain 的持久化执行层
- 专门负责处理应用的持久化执行功能，确保应用状态的持久化和可恢复性

简单来说，LangChain 是整体的开发框架，而 LangGraph 是其中专门负责持久化执行的技术组件，两者通常配合使用来构建更强大的 LLM 应用。

=== 完整对话历史 ===
共有 12 条消息:
1. [用户] 你好，请告诉我 LangChain 是什么？
2. [助手] 
你好！我来帮你搜索一下关于 LangChain 的信息。

3. [助手] LangChain 是一个用于开发 LLM 应用的框架
4. [助手] 

5. [助手] 未找到关于 'LangChain 框架 特点 用途' 的信息
6. [助手] 

7. [用户] 那 LangGraph 呢？
8. [助手] 

9. [助手] LangGraph 是 LangChain 的持久化执行层
10. [助手] 
用户询问了 LangChain 和 LangGraph，我已经通过搜索知识库获得了基本信息：

1. LangChain 是一个用于开发 LLM 应用的框架
2. LangGraph 是 LangC...
11. [助手] [18:13:07] 记录: 用户询问了 LangChain 和 LangGraph 的概念
12. [助手] 
根据我的知识库查询结果，为您介绍这两个概念：

**LangChain**
- LangChain 是一个用于开发 LLM 应用的框架
- 它为开发者提供了构建基于大语言模型的应用所需的工具和组件
...


In [8]:
# 总结学到的内容
learning_summary = {
    "Memory": {
        "概念": "代理能记住过去的交互",
        "工具": "checkpointer (MemorySaver)",
        "关键参数": "thread_id - 标识对话"
    },
    "Tools": {
        "概念": "让代理执行实际操作",
        "定义方式": "@tool 装饰器",
        "参数传递": "ToolRuntime 注入上下文"
    },
    "Context_Engineering": {
        "概念": "在正确的时间提供正确的信息",
        "实现方式": "动态系统提示、中间件",
        "好处": "让代理做出更好的决策"
    },
    "Graph_API": {
        "概念": "构建多步工作流",
        "组件": "节点、边、条件边",
        "用途": "定义复杂的流程和决策树"
    }
}

import json
print(json.dumps(learning_summary, indent=2, ensure_ascii=False))

{
  "Memory": {
    "概念": "代理能记住过去的交互",
    "工具": "checkpointer (MemorySaver)",
    "关键参数": "thread_id - 标识对话"
  },
  "Tools": {
    "概念": "让代理执行实际操作",
    "定义方式": "@tool 装饰器",
    "参数传递": "ToolRuntime 注入上下文"
  },
  "Context_Engineering": {
    "概念": "在正确的时间提供正确的信息",
    "实现方式": "动态系统提示、中间件",
    "好处": "让代理做出更好的决策"
  },
  "Graph_API": {
    "概念": "构建多步工作流",
    "组件": "节点、边、条件边",
    "用途": "定义复杂的流程和决策树"
  }
}
